In [ ]:
from qiskit import QuantumCircuit,execute,Aer
backend = Aer.get_backend('aer_simulator')

from random import randint
from math import sqrt

print("Done.")

In [ ]:
# We need data for classification 0, classification 1, and for new data to be classified as either classification 0 or classification 1.
train = {0:[],
         1:[]
        }
test = []

# Randomly assign lower integers to classification 0, higher integers to classification 1, and integers closer to classification 1 for the test state.
for i in range(0,8):
    train[0].append(randint(0,29))
    train[1].append(randint(70,99))
    test.append(randint(65,75))
    
# Convert all values to amplitudes, the sums of squares of which must equal 1.
sumtrain0 = sum(train[0])
sumtrain1 = sum(train[1])
sumtest = sum(test)
for i in range(0,8):
    train[0][i] = sqrt(train[0][i]/sumtrain0)
    train[1][i] = sqrt(train[1][i]/sumtrain1)
    test[i] = sqrt(test[i]/sumtest)
    
print(train)
print(test)

In [ ]:
# To compare 3-qubit states, we need 3 qubits for train data, 3 qubits for test data, and 1 ancilla qubit to compate the two states.
qc = QuantumCircuit(7,1)

def classify():
    initial_state = train[0] # Retrieve amplitudes for classification 0.
    qc.initialize(initial_state, [0,1,2]) # Map amplitudes to the first 3 qubits.
    initial_state = test # Retrieve the amplitudes for the test data.
    qc.initialize(initial_state, [3,4,5]) # Map amplitudes to the next 3 qubits.

    # SWAP Test, aka distance measures, aka kernel methods, aka inner products
    qc.h(6) # Apply a Hadamard to the ancilla qubit.
    for i in range(0,3):
        qc.cswap(6,i,i+3) # Using the ancilla as the control, SWAP like qubits of the train and test data.
    qc.h(6) # Apply another Hadamard to the ancilla qubit.
    qc.measure([6],[0]) # Measure only the ancilla qubit.
    
    job = execute(qc, backend, shots=10000)
    result = job.result()
    print(result.get_counts(qc))
    
    initial_state = train[1] # Retrieve amplitudes for classification 1.
    qc.initialize(initial_state, [0,1,2]) # Map amplitudes to the first 3 qubits.
    initial_state = test # Retrieve the amplitudes for the test data.
    qc.initialize(initial_state, [3,4,5]) # Map amplitudes to the next 3 qubits.
    qc.reset(6) # Reset the ancilla qubit for re-use. Initialize(), above, includes resets for the first 6 qubits.

    # SWAP Test, aka distance measures, aka kernel methods, aka inner products
    qc.h(6) # Apply a Hadamard to the ancilla qubit.
    for i in range(0,3):
        qc.cswap(6,i,i+3) # Using the ancilla as the control, SWAP like qubits of the train and test data.
    qc.h(6) # Apply another Hadamard to the ancilla qubit.
    qc.measure([6],[0]) # Measure only the ancilla qubit.
    
    job = execute(qc, backend, shots=10000)
    result = job.result()
    print(result.get_counts(qc))

print("Done.")

In [ ]:
for i in range(10):
    print(i)
    
    # Generate new test data each time.
    for i in range(0,8):
        test[i] = randint(65,75)
    sumtest = sum(test)
    for i in range(0,8):
        test[i] = sqrt(test[i]/sumtest)
        
    classify()
    
print("Done.")